In [41]:
import pandas as pd
data=pd.read_csv('../result/inte_target.tsv',sep='\t',index_col=0)
data.head()

,herb_id,herb_name,herb_type,target,Resource
0,HBIN041751,quinic acid,Aliphatic acid,CA2,Lianqiao
1,HBIN041751,quinic acid,Aliphatic acid,PTGS2,Lianqiao
2,HBIN041751,quinic acid,Aliphatic acid,GRIA2,Lianqiao
3,HBIN041751,quinic acid,Aliphatic acid,ACHE,Lianqiao
4,HBIN041751,quinic acid,Aliphatic acid,DPP4,Lianqiao


['LYZ_1xju.pdb',
 'MKI67_2aff.pdb',
 'LYZ_1xju.pdbqt',
 'CTSD_4obz.pdb',
 'HSPA5_3iuc.pdb',
 'IGHG1_6hyg.pdb',
 'S100A8_5hlv.pdb',
 'caffeic_acid.pdbqt']

In [70]:
energy_minimized

array([ 0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.371,  0.   , -0.371])

In [75]:
for protein in ['LYZ','MKI67','S100A8','CTSD','HSPA5','IGHG1']:
    pdb_files=[i for i in os.listdir('./') if 'pdb' in i]
    pdb_file_name=[i for i in pdb_files if protein in i][0]
    
    ingres=[i.replace(' ','_') for i in list(set(data.loc[data['target']==protein,'herb_name'].tolist()))]
    ingre_names=[i for i in ingres if f'{i}.sdf' in os.listdir('./')]
    for ingre in ingre_names:
        print(protein,ingre,'started')
        os.system(f'prepare_receptor -r {pdb_file_name} -o prepare/{protein}.pdbqt -A "hydrogens"')
        os.system(f'mk_prepare_ligand.py -i {ingre}.sdf -o {ingre}.pdbqt')
        os.system(f'pythonsh prepare_gpf.py -l {ingre}.pdbqt -r {ingre}.pdbqt -y -o prepare/{ingre}.gpf')
        
        with open(f'prepare/{ingre}.gpf','r') as f:
            for line in f.readlines():
                if 'gridcenter' in line:
                    print(line.split(' '))
                    loc=[float(line.split(' ')[1]),float(line.split(' ')[2]),float(line.split(' ')[3])]

        from vina import Vina


        v = Vina(sf_name='vina')

        v.set_receptor(f'prepare/{protein}.pdbqt')

        v.set_ligand_from_file(f'{ingre}.pdbqt')
        v.compute_vina_maps(center=loc, box_size=[100, 100, 100])

        # Score the current pose
        energy = v.score()
        print('Score before minimization: %.3f (kcal/mol)' % energy[0])

        # Minimized locally the current pose
        energy_minimized = v.optimize()
        print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
        v.write_pose(f'output/{protein}_{ingre}_minimized.pdbqt', overwrite=True)

        # Dock the ligand
        v.dock(exhaustiveness=32, n_poses=20)
        v.write_poses(f'output/{protein}_{ingre}.pdbqt', n_poses=5, overwrite=True)
        
        import pymol
        pymol.cmd.load(f'output/{protein}_{ingre}.pdbqt')
        pymol.cmd.load(f'prepare/{protein}.pdbqt')
        pymol.cmd.save(f'dock/{protein}_{ingre}.pdb')
        pymol.cmd.delete('all')
        
        os.system(f'mkdir dock/{protein}_{ingre}')
        os.system(f'plip -f dock/{protein}_{ingre}.pdb -xtyp -o dock/{protein}_{ingre}')
        print(protein,ingre,'finished')


LYZ vanillic_acid started
adding gasteiger charges to peptide
['gridcenter', '-0.112', '0.190', '0.019', '', '', '', '', '', '', '', '#', 'xyz-coordinates', 'or', 'auto\n']

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1      -0.4307          0          0
   2       -0.366      3.891      5.771
   3      -0.3643        4.5       6.44
   4      -0.3551      1.455      1.941
   5      -0.3361        6.2      9.529
   6      -0.3311      2.908        7.5
   7      -0.3309      2.491      4.032
   8      -0.3208      2.146      2.925
   9      -0.3186      2.564      2.907
  10      -0.3064      1.812      3.198
  11      -0.3015      3.338      4.807
  12      -0.3008      5.376      6.129
  13      -0.2993      2.283       7.85
  14      -0.2979      5.361      7.342
  15      -0.2789      4.357      4.642
  16      -0.2772      2.869      7.935
  17      -0.2673      3.926      4.733
  18      -0.2624      3

mkdir: dock/LYZ_vanillic_acid: File exists
2024-02-03 14:59:53,038 [INFO] [plipcmd.py:124] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.3.0
2024-02-03 14:59:53,038 [INFO] [plipcmd.py:125] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2024-02-03 14:59:53,039 [INFO] [plipcmd.py:126] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2024-02-03 14:59:53,039 [INFO] [plipcmd.py:49] plip.plipcmd: starting analysis of LYZ_vanillic_acid.pdb
Imagemagick not available. Images will not be resized or cropped.2024-02-03 14:59:53,306 [INFO] [plipcmd.py:165] plip.plipcmd: finished analysis, find the result files in dock/LYZ_vanillic_acid/


LYZ vanillic_acid finished
LYZ caffeic_acid started
adding gasteiger charges to peptide
['gridcenter', '0.182', '0.075', '0.033', '', '', '', '', '', '', '', '', '#', 'xyz-coordinates', 'or', 'auto\n']

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -5.688          0          0
   2       -5.631      20.75      22.39
   3       -5.522      1.647      4.095
   4        -5.43      27.17      28.05
   5       -5.378      24.45      25.48
   6       -5.352      28.11      29.27
   7       -5.344      34.14      34.83
   8       -5.284      26.46         27
   9        -5.26      33.76       34.7
  10       -5.256      27.26      28.63
  11       -5.253      27.89      29.05
  12        -5.23      33.82      34.89
  13       -5.204      33.93      34.73
  14       -5.195      34.03      35.13
  15       -5.181      27.45      28.31
  16       -5.049      20.41      21.87
  17       -5.001      34.14      3

mkdir: dock/LYZ_caffeic_acid: File exists


Computing Vina grid ... done.
Score before minimization: -0.457 (kcal/mol)
Score after minimization : -0.527 (kcal/mol)
Performing local search ... done.
Performing docking (random seed: 1968332197) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


...

IGHG1 emodin finished
IGHG1 pectolinarigenin started
adding gasteiger charges to peptide
['gridcenter', '-0.269', '0.034', '-0.008', '', '', '', '', '', '', '#', 'xyz-coordinates', 'or', 'auto\n']

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -7.453          0          0
   2       -7.085      7.919      12.01
   3       -7.034      1.138      3.188
   4       -6.874      1.254      3.219
   5       -6.823      40.43      41.97
   6       -6.674      25.84      26.94
   7       -6.622      38.88      40.46
   8       -6.588      1.527      6.269
   9       -6.579      7.054      9.604
  10       -6.442      8.955      12.16
  11       -6.383       2.26      6.158
  12       -6.356      8.264      11.72
  13       -6.338      22.58       23.6
  14       -6.294      7.396      10.47
  15       -6.279      2.307      3.833
  16        -6.26      34.86      37.05
  17       -6.258      39.39      40.83



0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


mkdir: dock/IGHG1_pectolinarigenin: File exists
2024-02-03 15:04:39,315 [INFO] [plipcmd.py:124] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.3.0
2024-02-03 15:04:39,315 [INFO] [plipcmd.py:125] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2024-02-03 15:04:39,315 [INFO] [plipcmd.py:126] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2024-02-03 15:04:39,315 [INFO] [plipcmd.py:49] plip.plipcmd: starting analysis of IGHG1_pectolinarigenin.pdb


IGHG1 pectolinarigenin finished


Imagemagick not available. Images will not be resized or cropped.2024-02-03 15:04:39,623 [INFO] [plipcmd.py:165] plip.plipcmd: finished analysis, find the result files in dock/IGHG1_pectolinarigenin/


In [77]:
for protein in ['LYZ','MKI67','S100A8','CTSD','HSPA5','IGHG1']:
    pdb_files=[i for i in os.listdir('./') if 'pdb' in i]
    pdb_file_name=[i for i in pdb_files if protein in i][0]
    
    ingres=[i.replace(' ','_') for i in list(set(data.loc[data['target']==protein,'herb_name'].tolist()))]
    ingre_names=[i for i in ingres if f'{i}.sdf' in os.listdir('./')]
    for ingre in ingre_names:
        print(protein,ingre,len(os.listdir(f'dock/{protein}_{ingre}')))

LYZ vanillic_acid 7
LYZ caffeic_acid 8
MKI67 daidzein 7
S100A8 rutin 8
CTSD quercetin 8
HSPA5 quercetin 8
IGHG1 irigenin 8
IGHG1 rosmarinic_acid 8
IGHG1 emodin 8
IGHG1 pectolinarigenin 8


In [ ]:
os.system(f'pythonsh prepare_gpf.py -l prepare/{ingre}.pdbqt -r prepare/{ingre}.pdbqt -y -o prepare/{ingre}.gpf')
        

In [65]:
ingre_names

[]

In [48]:
ingres=[i.replace(' ','_') for i in list(set(data.loc[data['target']=='LYZ','herb_name'].tolist()))]
[i for i in ingres if f'{i}.sdf' in os.listdir('./')]

['vanillic_acid', 'caffeic_acid']

In [53]:
import os
os.system('prepare_receptor -r LYZ_1xju.pdb -o prepare/LYZ_1xju.pdbqt -A "hydrogens"')

adding gasteiger charges to peptide


0

In [59]:
os.system('mk_prepare_ligand.py -i caffeic_acid.sdf -o prepare/caffeic_acid.pdbqt')

0

In [61]:
import os
os.system('pythonsh prepare_gpf.py -l prepare/caffeic_acid.pdbqt -r prepare/caffeic_acid.pdbqt -y -o prepare/caffeic_acid.gpf')

0

In [18]:
with open('caffeic_acid.gpf','r') as f:
    for line in f.readlines():
        if 'gridcenter' in line:
            print(line.split(' '))
            loc=[float(line.split(' ')[1]),float(line.split(' ')[2]),float(line.split(' ')[3])]

['gridcenter', '0.182', '0.075', '0.033', '', '', '', '', '', '', '', '', '#', 'xyz-coordinates', 'or', 'auto\n']


In [32]:
from vina import Vina


v = Vina(sf_name='vina')

v.set_receptor('LYZ_1xju.pdbqt')

v.set_ligand_from_file('caffeic_acid.pdbqt')
v.compute_vina_maps(center=loc, box_size=[20, 20, 20])

# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# Minimized locally the current pose
energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
v.write_pose('output/LYZ_1xju_caffeic_acid_minimized.pdbqt', overwrite=True)

# Dock the ligand
v.dock(exhaustiveness=32, n_poses=20)
v.write_poses('output/LYZ_1xju_caffeic_acid.pdbqt', n_poses=5, overwrite=True)


mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -4.123          0          0
   2       -4.044      8.662      10.43
   3       -3.984      11.71      12.77
   4       -3.956      2.973      3.971
   5       -3.896      2.713      6.396
   6       -3.878      12.77      14.15
   7       -3.847      9.505      10.52
   8       -3.847      4.172      5.884
   9        -3.84      8.098       8.89
  10       -3.831      8.152      10.24
  11       -3.828       8.64      10.31
  12       -3.796       8.21      10.02
  13         -3.7      4.969      6.912
  14       -3.601      8.691      10.63
  15       -3.582      10.39      12.24
  16       -3.579      1.194      5.947
  17       -3.578      11.69      12.97
  18       -3.569      9.805         11
  19       -3.564      1.514      5.934
  20       -3.561      8.891      9.904
Computing Vina grid ... Score before minimization: -0.457 (kcal/mol)
Score af

In [33]:
import pymol
pymol.cmd.load('output/LYZ_1xju_caffeic_acid.pdbqt')

In [34]:
pymol.cmd.load('LYZ_1xju.pdbqt')

In [35]:
pymol.cmd.save('dock/LYZ_1xju_caffeic_acid.pdb')

In [36]:
pymol.cmd.delete('all')

In [37]:
import os
os.system('mkdir dock/LYZ_1xju_caffeic_acid')
os.system('plip -f dock/LYZ_1xju_caffeic_acid.pdb -xtyp -o dock/LYZ_1xju_caffeic_acid')


mkdir: dock/LYZ_1xju_caffeic_acid: File exists
2024-02-03 14:15:51,031 [INFO] [plipcmd.py:124] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.3.0
2024-02-03 14:15:51,031 [INFO] [plipcmd.py:125] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2024-02-03 14:15:51,031 [INFO] [plipcmd.py:126] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2024-02-03 14:15:51,031 [INFO] [plipcmd.py:49] plip.plipcmd: starting analysis of LYZ_1xju_caffeic_acid.pdb
Imagemagick not available. Images will not be resized or cropped.2024-02-03 14:15:51,365 [INFO] [plipcmd.py:165] plip.plipcmd: finished analysis, find the result files in dock/LYZ_1xju_caffeic_acid/


0